In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
path_list = glob.glob('*')
path_list

['ccf_offline_stage1_test_revised.csv',
 'ccf_offline_stage1_train',
 'ccf_online_stage1_train',
 'df_offline.csv',
 'df_offline_predict.csv',
 'df_online.csv',
 'o2o优惠券使用预测_特征工程.ipynb',
 'sample_submission.csv',
 'xgb.model',
 '特征工程2.ipynb']

In [39]:
# 用户线下消费和优惠券领取行为
df_offline = pd.read_csv('./ccf_offline_stage1_train/ccf_offline_stage1_train.csv',sep = ',')
df_offline

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN
5,1439408,2632,NaN,NaN,0.0,NaN,20160516.0
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0
7,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
8,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
9,2029232,450,1532.0,30:5,0.0,20160530.0,NaN


In [3]:
# 从Date中提取出正负样本的标记
def change_Data(x):
    if np.isnan(x['Coupon_id']) and not np.isnan(x['Date']):
        return 0#负样本--普通样本
    elif not np.isnan(x['Coupon_id']) and np.isnan(x['Date']):
        return 2#普通样本--负样本
    elif not np.isnan(x['Coupon_id']) and not np.isnan(x['Date']):
        return 1#正样本

In [4]:
# Date_received当中时间的提取
def extract_time(x):
    if np.isnan(x):
        return [-10,-10,-10]
    else:
        x = str(x)
        x = x[:-2]
        year = x[:4]
        month = x[4:6]
        day = x[6:]
        return [int(year),int(month),int(day)]

In [5]:
# 优惠率：x \in [0,1]代表折扣率；x:y表示满x减y。单位是元
def discount_rate_ratio(x):
    x = str(x)
    if x == 'fixed':
        return -10
    if ':' not in x:
        x = float(x)
        if x >= 0 and x <= 1:
            return x
        else:
            return -10
    else:
        return -10 

In [6]:
# 获取满x减y的x            
def discount_rate_x(x):
    x = str(x)
    if ':' in str(x):
        list_x = x.split(':')
        return int(list_x[0])
    else:
        return -1

In [7]:
# 获取满x减y的y
def discount_rate_y(x):
    x = str(x)
    if ':' in str(x):
        list_x = x.split(':')
        return int(list_x[1])
    else:
        return -10

In [8]:
# 将Distance里面的异常值去掉
def distance(x):
    if np.isnan(x):
        return -10
    else:
        return float(x)

In [9]:
# 添加是否有优惠券的标记
def has_Coupen2(x):
    if isinstance(x, float):
        if np.isnan(x):
            return 0
        else:
            return 1

In [296]:
# 测试怎么使用 ,原来的数据集太大了，速度慢，选取一小部分来测试    
df0 = df_offline.iloc[[i for i in range(100,200,1)]]
time_about =(list(df0['Date_received'].apply(extract_time)))
# s1 = ([x[1] for x in time_about])
# print(s1)
df0['year'] = [x[0] for x in time_about]
df0['month'] = [x[1] for x in time_about]
df0['day'] = [x[2] for x in time_about]
# df0['samples_class'] = df0.apply(change_Data, axis = 1)
# df0['discount_rate_x'] = df0['Discount_rate'].apply(discount_rate_x)
# df0['discount_rate_y'] = df0['Discount_rate'].apply(discount_rate_y)
# df0['discount_rate_ratio'] = df0['Discount_rate'].apply(discount_rate_ratio)
# df0['Distance'] = df0['Distance'].apply(distance)
df0

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,year,month,day
100,114747,3381,9776.0,10:5,0.0,20160129.0,20160221.0,2016,1,29
101,114747,4833,13628.0,200:20,1.0,20160130.0,NaN,2016,1,30
102,114747,5341,NaN,NaN,0.0,NaN,20160621.0,-10,-10,-10
103,114747,5341,11539.0,30:5,0.0,20160128.0,NaN,2016,1,28
104,245878,760,2418.0,30:5,0.0,20160601.0,NaN,2016,6,1
105,3409760,5341,7751.0,50:10,1.0,20160128.0,NaN,2016,1,28
106,3409760,6901,2366.0,30:5,7.0,20160530.0,NaN,2016,5,30
107,3409760,7555,17.0,30:5,10.0,20160126.0,NaN,2016,1,26
108,3409760,450,8555.0,30:5,10.0,20160126.0,NaN,2016,1,26
109,3409760,5341,111.0,30:5,1.0,20160127.0,NaN,2016,1,27


In [40]:
df_offline2 = df_offline
df_offline2['samples_class'] = df_offline2.apply(change_Data, axis = 1)
df_offline2['has_Coupon'] = df_offline2['Coupon_id'].apply(has_Coupen)
df_offline2['discount_rate_x'] = df_offline2['Discount_rate'].apply(discount_rate_x)
df_offline2['discount_rate_y'] = df_offline2['Discount_rate'].apply(discount_rate_y)
df_offline2['discount_rate_ratio'] = df_offline2['Discount_rate'].apply(discount_rate_ratio)
df_offline2['Distance'] = df_offline2['Distance'].apply(distance)

In [41]:
time_get =list(df_offline2['Date_received'].apply(extract_time))
df_offline2['get_year'] = [x[0] for x in time_get]
df_offline2['get_month'] = [x[1] for x in time_get]
df_offline2['get_day'] = [x[2] for x in time_get]
time_use =list(df_offline2['Date'].apply(extract_time))
df_offline2['use_year'] = [x[0] for x in time_use]
df_offline2['use_month'] = [x[1] for x in time_use]
df_offline2['use_day'] = [x[2] for x in time_use]

In [42]:
df_offline2['has_Coupon'] = df_offline2['Coupon_id'].apply(has_Coupen2)

In [43]:
offline_list2 = list(df_offline2.columns)
print(offline_list2)
df_offline2 = df_offline2[['User_id','Coupon_id','Distance','discount_rate_x','discount_rate_y',
                            'discount_rate_ratio','get_year','get_month','get_day','use_year',
                            'use_month','use_day','has_Coupon','samples_class']]
df_offline2.to_csv('df_offline.csv', index = False)

['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received', 'Date', 'samples_class', 'has_Coupon', 'discount_rate_x', 'discount_rate_y', 'discount_rate_ratio', 'get_year', 'get_month', 'get_day', 'use_year', 'use_month', 'use_day']


In [39]:
df_offline2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 14 columns):
User_id                int64
Coupon_id              float64
Distance               float64
discount_rate_x        int64
discount_rate_y        int64
discount_rate_ratio    float64
get_year               int64
get_month              int64
get_day                int64
use_year               int64
use_month              int64
use_day                int64
has_Coupon             float64
samples_class          int64
dtypes: float64(4), int64(10)
memory usage: 187.4 MB


In [313]:
?df01.to_csv

In [44]:
df01 = pd.read_csv('df_offline.csv',sep = ',')
df01

,User_id,Coupon_id,Distance,discount_rate_x,discount_rate_y,discount_rate_ratio,get_year,get_month,get_day,use_year,use_month,use_day,has_Coupon,samples_class
0,1439408,NaN,0.0,-1,-10,-10.0,-10,-10,-10,2016,2,17,0,0
1,1439408,11002.0,1.0,150,20,-10.0,2016,5,28,-10,-10,-10,1,2
2,1439408,8591.0,0.0,20,1,-10.0,2016,2,17,-10,-10,-10,1,2
3,1439408,1078.0,0.0,20,1,-10.0,2016,3,19,-10,-10,-10,1,2
4,1439408,8591.0,0.0,20,1,-10.0,2016,6,13,-10,-10,-10,1,2
5,1439408,NaN,0.0,-1,-10,-10.0,-10,-10,-10,2016,5,16,0,0
6,1439408,8591.0,0.0,20,1,-10.0,2016,5,16,2016,6,13,1,1
7,1832624,7610.0,0.0,200,20,-10.0,2016,4,29,-10,-10,-10,1,2
8,2029232,11951.0,1.0,200,20,-10.0,2016,1,29,-10,-10,-10,1,2
9,2029232,1532.0,0.0,30,5,-10.0,2016,5,30,-10,-10,-10,1,2


In [21]:
#  用户线上点击/消费和优惠券领取行为
df_online = pd.read_csv('./ccf_online_stage1_train/ccf_online_stage1_train.csv', sep = ',')
# df_online.loc[[i for i in range(100,200,1)]]
df_online.groupby('Action')[['Date','Coupon_id']].count()

,Date,Coupon_id
Action,,
0,9401780,0
1,1372148,216459
2,0,655898


In [10]:
# 从Date中提取出正负样本的标记
# Coupon_id是str/float的格式，Date是float的格式
def change_Data1(x):
#     print(type(x['Coupon_id']),type(x['Date']))
    if isinstance(x['Coupon_id'], float):
        if not np.isnan(x['Coupon_id']) and np.isnan(x['Date']):
            return -1
        elif np.isnan(x['Coupon_id']) and not np.isnan(x['Date']):
            return 0  
    elif isinstance(x['Coupon_id'], str):
        if not np.isnan(x['Date']) :
            return 1
        elif np.isnan(x['Date']):
            return -1
        elif x['Coupon_id'] == 'fixed':
            return -10

In [11]:
# 添加限时优惠的标记
def add_fixed(x):
#     print(type(x['Discount_rate']),type(x['Coupon_id']))
    if isinstance(x['Discount_rate'], str) and isinstance(x['Coupon_id'], str):
        if 'fixed' == x['Discount_rate']:
            return 1
        else:
            return -10
    else:
        return -10

In [12]:
# 添加是否有优惠券的标记
def has_Coupen(x):
    if isinstance(x, float):
        if np.isnan(x):
            return 0
    else:
        return 1


In [22]:
# 测试怎么使用 ,原来的数据集太大了，速度慢，选取一小部分来测试    
df11 = df_online.loc[[i for i in range(0,100,1)]]
df11

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513.0,NaN
1,13740231,34805,1,NaN,NaN,NaN,20160321.0
2,14336199,18907,0,NaN,NaN,NaN,20160618.0
3,14336199,18907,0,NaN,NaN,NaN,20160618.0
4,14336199,18907,0,NaN,NaN,NaN,20160618.0
5,14336199,18907,0,NaN,NaN,NaN,20160618.0
6,14336199,18907,0,NaN,NaN,NaN,20160618.0
7,14336199,18907,0,NaN,NaN,NaN,20160618.0
8,14336199,18907,0,NaN,NaN,NaN,20160618.0
9,14336199,18907,0,NaN,NaN,NaN,20160618.0


In [24]:
# 使用小数据集测试效果，速度快
df11['samples_class'] = df11.apply(change_Data1, axis = 1)
df11['has_Coupon'] = df11['Coupon_id'].apply(has_Coupen)
time_get =list(df11['Date_received'].apply(extract_time))
df11['get_year'] = [x[0] for x in time_get]
df11['get_month'] = [x[1] for x in time_get]
df11['get_day'] = [x[2] for x in time_get]
time_use =list(df11['Date'].apply(extract_time))
df11['use_year'] = [x[0] for x in time_use]
df11['use_month'] = [x[1] for x in time_use]
df11['use_day'] = [x[2] for x in time_use]
df11['discount_rate_x'] = df11['Discount_rate'].apply(discount_rate_x)
df11['discount_rate_y'] = df11['Discount_rate'].apply(discount_rate_y)
df11['discount_rate_fixed'] = df11.apply(add_fixed, axis = 1)
df11['discount_rate_ratio'] = df11['Discount_rate'].apply(discount_rate_ratio)

df11

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,samples_class,has_Coupon,get_year,get_month,get_day,use_year,use_month,use_day,discount_rate_x,discount_rate_y,discount_rate_fixed,discount_rate_ratio
0,13740231,18907,2,100017492,500:50,20160513.0,NaN,-1,1,2016,5,13,-10,-10,-10,500,50,-10,-10
1,13740231,34805,1,NaN,NaN,NaN,20160321.0,0,0,-10,-10,-10,2016,3,21,-1,-10,-10,-10
2,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10
3,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10
4,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10
5,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10
6,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10
7,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10
8,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10
9,14336199,18907,0,NaN,NaN,NaN,20160618.0,0,0,-10,-10,-10,2016,6,18,-1,-10,-10,-10


In [12]:
df_online2 = df_online
df_online2['samples_class'] = df_online2.apply(change_Data1, axis = 1)
time_get =list(df_online2['Date_received'].apply(extract_time))
df_online2['get_year'] = [x[0] for x in time_get]
df_online2['get_month'] = [x[1] for x in time_get]
df_online2['get_day'] = [x[2] for x in time_get]
time_use =list(df_online2['Date'].apply(extract_time))
df_online2['use_year'] = [x[0] for x in time_use]
df_online2['use_month'] = [x[1] for x in time_use]
df_online2['use_day'] = [x[2] for x in time_use]

In [13]:
df_online2['discount_rate_x'] = df_online2['Discount_rate'].apply(discount_rate_x)
df_online2['discount_rate_y'] = df_online2['Discount_rate'].apply(discount_rate_y)
df_online2['discount_rate_fixed'] = df_online2.apply(add_fixed, axis = 1)
df_online2['discount_rate_ratio'] = df_online2['Discount_rate'].apply(discount_rate_ratio)

In [14]:
df_online2 = df_online2[['User_id','Coupon_id','Action','discount_rate_x','discount_rate_y',
                         'discount_rate_ratio','discount_rate_fixed','get_year','get_month',
                         'get_day','use_year','use_month','use_day','samples_class']]
df_online2.to_csv('df_online.csv', index = False)

In [25]:
df03 = pd.read_csv('df_online.csv')
print(df03.shape)
df03['samples_class'].value_counts()
# df03

(11429826, 14)


 0    10557469
-1      655898
 1      216459
Name: samples_class, dtype: int64

In [27]:
df03['has_Coupon'] = df03['Coupon_id'].apply(has_Coupen)
df03 = df03[['User_id','Coupon_id','Action','discount_rate_x','discount_rate_y',
                         'discount_rate_ratio','discount_rate_fixed','get_year','get_month',
                         'get_day','use_year','use_month','use_day','has_Coupon','samples_class']]
df03.to_csv('df_online.csv', index = False)

In [13]:
# 用户O2O线下优惠券使用预测样本
df_offline_predict= pd.read_csv('ccf_offline_stage1_test_revised.csv', sep = ',')
df_offline_predict

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708
5,4005121,450,9983,30:5,0.0,20160706
6,4347394,450,9983,30:5,0.0,20160716
7,3094273,760,13602,30:5,1.0,20160727
8,5139970,450,9983,30:5,10.0,20160729
9,3237121,760,13602,30:5,1.0,20160703


In [14]:
# Date_received当中时间的提取
def extract_time2(x):
    if np.isnan(x):
        return [-10,-10,-10]
    else:
        x = str(x)
        year = x[:4]
        month = x[4:6]
        day = x[6:]
        return [int(year),int(month),int(day)]

In [15]:
df_offline_predict2 = df_offline_predict
time_get =list(df_offline_predict2['Date_received'].apply(extract_time2))
df_offline_predict2['get_year'] = [x[0] for x in time_get]
df_offline_predict2['get_month'] = [x[1] for x in time_get]
df_offline_predict2['get_day'] = [x[2] for x in time_get]

In [16]:
df_offline_predict2['discount_rate_x'] = df_offline_predict2['Discount_rate'].apply(discount_rate_x)
df_offline_predict2['discount_rate_y'] = df_offline_predict2['Discount_rate'].apply(discount_rate_y)
df_offline_predict2['discount_rate_ratio'] = df_offline_predict2['Discount_rate'].apply(discount_rate_ratio)

In [17]:
df_offline_predict2 = df_offline_predict2[['User_id','Coupon_id','Distance','Date_received','discount_rate_x','discount_rate_y',
                         'discount_rate_ratio','get_year','get_month','get_day',]]
df_offline_predict2

,User_id,Coupon_id,Distance,Date_received,discount_rate_x,discount_rate_y,discount_rate_ratio,get_year,get_month,get_day
0,4129537,9983,1.0,20160712,30,5,-10.0,2016,7,12
1,6949378,3429,NaN,20160706,30,5,-10.0,2016,7,6
2,2166529,6928,5.0,20160727,200,20,-10.0,2016,7,27
3,2166529,1808,5.0,20160727,100,10,-10.0,2016,7,27
4,6172162,6500,2.0,20160708,30,1,-10.0,2016,7,8
5,4005121,9983,0.0,20160706,30,5,-10.0,2016,7,6
6,4347394,9983,0.0,20160716,30,5,-10.0,2016,7,16
7,3094273,13602,1.0,20160727,30,5,-10.0,2016,7,27
8,5139970,9983,10.0,20160729,30,5,-10.0,2016,7,29
9,3237121,13602,1.0,20160703,30,5,-10.0,2016,7,3


In [18]:
df_offline_predict2['has_Coupon'] = df_offline_predict2['Coupon_id'].apply(has_Coupen)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_offline_predict2.to_csv('df_offline_predict.csv', index = False)

In [20]:
df_offline_predict2 = pd.read_csv('df_offline_predict.csv')
df_offline_predict2['Distance'] = df_offline_predict2['Distance'].apply(distance)
df_offline_predict2.to_csv('df_offline_predict.csv', index = False)